# İmport libraries

In [ ]:
import pandas as pd
import numpy as np
# Path of the file to read
iowa_file_path = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'

df = pd.read_csv(iowa_file_path)

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.machine_learning.ex3 import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn import metrics
from sklearn import linear_model
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge


In [ ]:
df.head()

# 2.The discovery and analysis function

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
len(df)

# 3.Split the DataFrame into numeric and categorical columns


In [ ]:
numeric_columns = df.select_dtypes(include=['number'])  # Select columns with numeric data types
categorical_columns = df.select_dtypes(include=['object'])  # Select columns with object (string) data types


In [ ]:
print(len(df.columns),'||Sum of numeric and object columns>',len(numeric_columns.columns)+len(categorical_columns.columns))

# Numeric columns

In [ ]:
numeric_columns.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

In [ ]:
#We'll use the KNNImputer class from scikit-learn to fill NAN values
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
df_imputed = imputer.fit_transform(numeric_columns)
numeric_columns = pd.DataFrame(df_imputed, columns=numeric_columns.columns)
numeric_columns.isnull().sum()

# Catagoric columns

In [ ]:
categorical_columns.head()

In [ ]:
categorical_columns.isnull().sum()

In [ ]:
len(categorical_columns)

In [ ]:
columns_to_drop = ['MiscFeature', 'Fence', 'PoolQC', 'Alley']
categorical_columns.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
categorical_columns.head()

In [ ]:
# Initialize the SimpleImputer with the "most_frequent" strategy
imputer = SimpleImputer(strategy='most_frequent')
categorical_columns_imputed = imputer.fit_transform(categorical_columns)
categorical_columns = pd.DataFrame(categorical_columns_imputed, columns=categorical_columns.columns)

print(categorical_columns.isnull().sum())

In [ ]:
for column in categorical_columns.columns:
    if pd.api.types.is_categorical_dtype(categorical_columns[column]):
        unique_values = categorical_columns[column].cat.categories
    else:
        unique_values = categorical_columns[column].unique()
    print(f"Unique:'{column}'> {unique_values}")

In [ ]:
# Initialize a dictionary to store LabelEncoders for each column
categorical_column_names = categorical_columns.columns

label_encoders = {}

for column in categorical_column_names:
    encoder = LabelEncoder()
    categorical_columns[column] = encoder.fit_transform(categorical_columns[column])
    label_encoders[column] = encoder
categorical_columns.head()

In [ ]:
y = numeric_columns.pop('SalePrice')

****Power Tranformation

In [ ]:
PTer = PowerTransformer(method='yeo-johnson') 
transformed_data = PTer.fit_transform(numeric_columns)
numeric_columns = pd.DataFrame(transformed_data, columns=numeric_columns.columns)
numeric_columns.head(3)

# Concat Data

In [ ]:
x = pd.concat([categorical_columns, numeric_columns], axis=1)
x.head()

# Visulation

In [ ]:
x.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

In [ ]:
print(y.describe())
plt.figure(figsize=(9, 8))
sns.distplot(y, color='g', bins=100, hist_kws={'alpha': 0.4});

# Buliding Model

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [ ]:
from sklearn import linear_model
lr=linear_model.LinearRegression()
lr.fit(X_train,Y_train)

In [ ]:
s=lr.predict(X_train)

In [ ]:
print('MAE:',metrics.mean_absolute_error(Y_train,s))
print('RMSE:',np.sqrt(metrics.mean_squared_error(Y_train,s)))
print('R_squared:',metrics.r2_score(Y_train,s))

In [ ]:
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(X_train,Y_train)
rf_val_predictions = rf_model.predict(X_test)
rf_val_mae = mean_absolute_error(rf_val_predictions, Y_test)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

In [ ]:
# To improve accuracy, create a new Random Forest model which you will train on all training data
rf_model_on_full_data = RandomForestRegressor()

# fit rf_model_on_full_data on all data from the training data
rf_model_on_full_data.fit(X_train, Y_train)

In [ ]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
target = train['SalePrice']
test_ids = test['Id']

train = train.drop(['SalePrice', 'Id'], axis = 1)
test = test.drop(['Id'], axis = 1)

dataset = pd.concat([train, test], axis = 0)
dataset


In [ ]:
dataset.select_dtypes('object').loc[:, dataset.isna().sum()> 0].columns

In [ ]:
categorical_with_none = [
    'Alley',
    'MasVnrType',
    'BsmtQual',
    'BsmtCond',
    'BsmtExposure',
    'BsmtFinType1',
    'BsmtFinType2',
    'FireplaceQu',
    'GarageType',
    'GarageFinish',
    'GarageQual',
    'GarageCond',
    'PoolQC',
    'Fence',
    'MiscFeature',   
]
categorical = dataset.select_dtypes('object').loc[:, dataset.isna().sum() > 0].columns
categorical

In [ ]:
for col in categorical_with_none:
    dataset[col] = dataset[col].fillna('None')
for col in categorical:
    mod = dataset[col].mode()[0]
    dataset[col] = dataset[col].fillna(mod)

In [ ]:
dataset.select_dtypes('object').loc[:, dataset.isna().sum()> 0].columns

In [ ]:
dataset.select_dtypes(np.number).loc[:, dataset.isna().sum() > 0].columns

In [ ]:
def FillWithKNN(dataset, ToFill):
    df = dataset.copy()

    numeric_df = df.select_dtypes(np.number)
    non_na_columns = numeric_df.loc[:, numeric_df.isna().sum() == 0].columns
    y_train = numeric_df.loc[numeric_df[ToFill].isna() == False, ToFill]
    X_train = numeric_df.loc[numeric_df[ToFill].isna() == False, non_na_columns]
    X_test = numeric_df.loc[numeric_df[ToFill].isna() == True, non_na_columns]
    
    knn = KNeighborsRegressor(n_neighbors = 10)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)

    df.loc[df[ToFill].isna() == True, ToFill] = y_pred
    return df
    

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
na_columns = dataset.columns[(dataset.isna().sum()) > 0]
for column in na_columns:
    dataset = FillWithKNN(dataset, column)

In [ ]:
print(train.shape, test.shape, dataset.shape)
train = dataset.iloc[:train.shape[0], :].copy()
test = dataset.iloc[train.shape[0]:, :].copy()
full = [train, test]
print(train.shape, test.shape, dataset.shape)

In [ ]:
for ds in full:
    categories = ds.select_dtypes('object').columns
    print(len(categories))
    for column in categories:
        dic = pd.concat([ds[column], target], axis = 1).groupby(column).mean().sort_values(by = 'SalePrice')
        mp = {}
        p = 0
        for it in dic.index:
            mp[it] = p
            p += 1
        ds[column] = ds[column].map(mp)
    categories = ds.select_dtypes('object').columns
    print(len(categories))

In [ ]:
X = train.copy()
y = target.copy()
X = StandardScaler().fit_transform(X)
test = StandardScaler().fit_transform(test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 8, shuffle = True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
acc = {}

In [ ]:
DT_reg = RandomForestRegressor(n_estimators = 100)
DT_reg.fit(X_train, y_train)
pred = DT_reg.predict(X_test)
DT_TestAccuracy = mean_squared_error(pred, y_test)
print("Decision Tree Testing Accuracy: ", round(DT_TestAccuracy, 2))
acc['DecisionTree'] = DT_TestAccuracy

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
pred = lin_reg.predict(X_test)
lin_TestAccuracy = mean_squared_error(pred, y_test)
print("Linear Regression Testing Accuracy: ", round(lin_TestAccuracy, 2))
acc['Linear Regression'] = lin_TestAccuracy

In [ ]:
ridge_reg = Ridge(alpha = 0.01)
ridge_reg.fit(X_train, y_train)
pred = ridge_reg.predict(X_test)
Ridge_TestAccuracy = mean_squared_error(pred, y_test)
print("Ridge Regression Testing Accuracy: ", round(Ridge_TestAccuracy, 2))
acc['Ridge'] = Ridge_TestAccuracy

In [ ]:
y_pred = DT_reg.predict(test)

In [ ]:
submission = pd.DataFrame({
        "Id": range(1461, 2920),
        "SalePrice": y_pred
    })

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index = False)


In [ ]:
submission